# Level Break Out

### Load the data

In [11]:
import numpy as np
import pandas as pd
from itertools import combinations
import os
from multiprocessing import Pool
from functools import partial
import threading
import datetime
from pytz import timezone
import requests
from scipy.stats import linregress
import itertools
import logging
import os

In [29]:
df = pd.read_csv("SUPREMEIND.BO.csv")
stockname = 'INFY'
# df=df[df['volume']!=0]
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

# df['EMA'] = ta.ema(df.Close, length=150)
# df.tail()

# df=df[2000:10000]

In [117]:
window=21
percentage = 1
pivot_line_count  = 3 #3
alert_file_name = "alerts.xlsx"

### Trend detection

In [118]:
# EMAsignal = [0]*len(df)
# backcandles = 15

# for row in range(backcandles, len(df)):
#     upt = 1
#     dnt = 1
#     for i in range(row-backcandles, row+1):
#         if max(df.Open[i], df.Close[i])>=df.EMA[i]:
#             dnt=0
#         if min(df.Open[i], df.Close[i])<=df.EMA[i]:
#             upt=0
#     if upt==1 and dnt==1:
#         EMAsignal[row]=3
#     elif upt==1:
#         EMAsignal[row]=2
#     elif dnt==1:
#         EMAsignal[row]=1

# df['EMASignal'] = EMAsignal

In [119]:
def isPivot(candle, window):
    """
    function that detects if a candle is a pivot/fractal point
    args: candle index, window before and after candle to test if pivot
    returns: 1 if pivot High, 2 if pivot Low, 3 if both and 0 default
    """
    if candle-window < 0 or candle+window >= len(df):
        return 0
    
    pivotHigh = 1
    pivotLow = 2
    for i in range(candle-window, candle+window+1):
        if df.iloc[candle].Low > df.iloc[i].Low:
            pivotLow=0
        if df.iloc[candle].High < df.iloc[i].High:
            pivotHigh=0
    if (pivotHigh and pivotLow):
        return 3
    elif pivotHigh:
        return pivotHigh
    elif pivotLow:
        return pivotLow
    else:
        return 0

In [120]:
df['isPivot'] = df.apply(lambda x: isPivot(x.name,window), axis=1)

In [121]:
def plus_minus_01_percent(combination,percentage):
    x_values,y_values = zip(*combination)
    slope, intercept, _, _, _ = linregress(x_values[0:2], y_values[0:2])
    predicted_y_value = slope * x_values[-1] + intercept
    actual_y_value = y_values[-1]
    # print(x_values[-1],y_values[-1],x_values[0:2], y_values[0:2])
    # Calculate 0.1% of the value
    # percent_01 = percentage / 100 * actual_y_value  # 0.1

    percent_difference = abs(predicted_y_value - actual_y_value) / actual_y_value * 100
    # print(percent_difference,percentage)
    return slope, intercept,percent_difference <= percentage

    print(slope, intercept,plus_value,minus_value,actual_y_values,predicted_y_values)
    print(predicted_y_values>minus_value and predicted_y_values < plus_value)
    return slope, intercept,predicted_y_values>minus_value and predicted_y_values < plus_value


def detect_structure(stockname,candle, backcandles, window):
    # print(candle)
    # return
    """
    Attention! window should always be greater than the pivot window! to avoid look ahead bias
    """
    if (candle <= (backcandles+window)) and (candle+window+1 >= len(df)):
        # print(candle)
        return 0
    # if(candle < 5818):
        # return 0
    # print(candle) 
    
    # localdf = df.iloc[candle-backcandles-window:candle-window] 
    localdf = df.iloc[0:candle-window] 

    Highs = localdf[localdf['isPivot'] == 1].High.tail(5).values
    x_values_Highs = localdf[localdf['isPivot'] == 1].index[-5:].values
    Highs_values = pd.DataFrame(localdf[localdf['isPivot'] == 1].High.tail(5))

    Lows = localdf[localdf['isPivot'] == 2].Low.tail(5).values
    x_values_Lows = localdf[localdf['isPivot'] == 2].index[-5:].values
    Lows_values = pd.DataFrame(localdf[localdf['isPivot'] == 2].Low.tail(5))
    levelbreak = 0
    global alerts
    if Lows_values.shape[0]>=pivot_line_count and candle-window-1 == Lows_values.index[-1]:
        combinations = list(itertools.combinations(zip(x_values_Lows, Lows), pivot_line_count))
        leatest_combinations = [[(index_low[0], index_low[1]) for index_low in combination] for combination in combinations]
        for combination in leatest_combinations:
            # if(plus_minus_01_percent(combination.index.values,combination['Low'].values,percentage)):
            slope, intercept , is_line= plus_minus_01_percent(combination,percentage)
            # print(candle,combination,is_line,1)
            if(is_line):
                # print("Low",combination)
                levelbreak = 1
                # alert = pd.DataFrame(combination).copy(deep=True)
                alert = pd.DataFrame(combination, columns=['index', 'value']).set_index('index').copy(deep=True)
                # print(alert)
                row_to_append = pd.DataFrame({
                    'stockname' : stockname,
                    'alert_date' : [df.iloc[candle].Datetime],
                    'rowNumber' : candle,                    
                    'date1': [df.iloc[alert.index[0]].Datetime],
                    'row1': [alert.index[0]],
                    'value1': [alert.iloc[0, 0]],
                    'date2': [df.iloc[alert.index[1]].Datetime],
                    'row2': [alert.index[1]],
                    'value2': [alert.iloc[1, 0]],
                    'date3': [df.iloc[alert.index[2]].Datetime] if pivot_line_count>2 else 0,
                    'row3': [alert.index[2]] if pivot_line_count>2 else 0,
                    'value3': [alert.iloc[2, 0]] if pivot_line_count>2 else 0,
                    'buyORsell' : 'Low' if levelbreak==1 else 'High',
                    "slope" : slope,
                    "intercept" : intercept,
                    "window_size":window,
                    "percentage_value" : percentage,
                    "pivot_line_count" : pivot_line_count
                })
                # global alerts
                alerts = pd.concat([alerts, row_to_append], ignore_index=True)
                # print(alerts)
    if Highs_values.shape[0]>=pivot_line_count and candle-window-1 == Highs_values.index[-1]:
        combinations = list(itertools.combinations(zip(x_values_Highs, Highs), pivot_line_count))
        leatest_combinations = [[(index_high[0], index_high[1]) for index_high in combination] for combination in combinations]
        # leatest_combinations = list(filter(lambda x: x not in high_pivot_all_combinations, leatest_combinations))
        for combination in leatest_combinations:
            # if(plus_minus_01_percent(combination.index.values,combination['Low'].values,percentage)):
            slope, intercept , is_line= plus_minus_01_percent(combination,percentage)
            # print(combination)
            if(is_line):
                levelbreak = 2
                # alert = Highs_values.copy(deep=True)
                alert = pd.DataFrame(combination, columns=['index', 'value']).set_index('index').copy(deep=True)
                # print(alert)
                row_to_append = pd.DataFrame({
                    'stockname' : stockname,
                    'alert_date' : [df.iloc[candle].Datetime],
                    'rowNumber' : candle,
                    'date1': [df.iloc[alert.index[0]].Datetime],
                    'row1': [alert.index[0]],
                    'value1': [alert.iloc[0, 0]],
                    'date2': [df.iloc[alert.index[1]].Datetime],
                    'row2': [alert.index[1]],
                    'value2': [alert.iloc[1, 0]],
                    'date3': [df.iloc[alert.index[2]].Datetime] if pivot_line_count>2 else 0,
                    'row3': [alert.index[2]] if pivot_line_count>2 else 0,
                    'value3': [alert.iloc[2, 0]] if pivot_line_count>2 else 0,
                    'buyORsell' : 'Low' if levelbreak==1 else 'High',
                    "slope" : slope,
                    "intercept" : intercept,
                    "window_size":window,
                    "percentage_value" : percentage,
                    "pivot_line_count" : pivot_line_count
                })
                # global alerts
                alerts = pd.concat([alerts, row_to_append], ignore_index=True)
                # print(alerts)

    alerts = alerts.drop_duplicates()
    return levelbreak


In [122]:
combinations = [
                    [(5844, 1499.0), (5880, 1519.300049), (5896, 1518.5)]
            ]
for combination in combinations:
    print(combination,plus_minus_01_percent(combination=combination,percentage=percentage))

[(5844, 1499.0), (5880, 1519.300049), (5896, 1518.5)] (0.5638902499999985, -1796.374620999991, True)


In [123]:
isExist = os.path.exists(alert_file_name)
if(isExist):
    alerts = pd.read_excel(alert_file_name)
else:
    alerts = pd.DataFrame()

In [124]:
#df['pattern_detected'] = df.index.map(lambda x: detect_structure(x, backcandles=40, window=15))
# alerts = pd.DataFrame(columns=['stockname','rowNumber','alert_date','row1', 'date1', 'value1', 'row2', 'date2', 'value2','row3', 'date3', 'value3','buyORsell'])
low_pivot_all_combinations = []
high_pivot_all_combinations = []
df['pattern_detected'] = df.apply(lambda row: detect_structure(stockname,
                                                               row.name, backcandles=20, window=window,
                                                            #    low_pivot_all_combinations=low_pivot_all_combinations,
                                                            #    high_pivot_all_combinations = high_pivot_all_combinations,
                                                                # alerts=alerts,
                                                               ), axis=1)
alerts = alerts.drop_duplicates()

In [125]:
alerts

,stockname,alert_date,rowNumber,date1,row1,value1,date2,row2,value2,date3,row3,value3,buyORsell,slope,intercept,window_size,percentage_value,pivot_line_count
0,INFY,02-12-2005 00:00:00,709,17-05-2004 00:00:00,342,64.46,14-07-2005 00:00:00,615,132.83,28-10-2005 00:00:00,687,150.00,Low,0.250440,-21.190330,21,1,3
1,INFY,21-02-2006 00:00:00,763,29-04-2005 00:00:00,561,117.28,14-07-2005 00:00:00,615,132.83,18-01-2006 00:00:00,741,168.76,Low,0.287963,-44.267222,21,1,3
2,INFY,12-05-2006 00:00:00,816,11-10-2005 00:00:00,675,170.63,09-01-2006 00:00:00,735,192.50,07-04-2006 00:00:00,794,215.56,High,0.364500,-75.407500,21,1,3
3,INFY,14-07-2006 00:00:00,861,29-04-2005 00:00:00,561,117.28,14-07-2005 00:00:00,615,132.83,18-01-2006 00:00:00,741,168.76,Low,0.287963,-44.267222,21,1,3
4,INFY,14-08-2006 00:00:00,882,11-10-2005 00:00:00,675,170.63,09-01-2006 00:00:00,735,192.50,07-04-2006 00:00:00,794,215.56,High,0.364500,-75.407500,21,1,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
74,INFY,28-10-2022 00:00:00,4883,29-09-2021 00:00:00,4615,1655.00,27-01-2022 00:00:00,4697,1665.00,28-02-2022 00:00:00,4719,1665.00,Low,0.121951,1092.195122,21,1,3
75,INFY,14-03-2023 00:00:00,4977,17-01-2022 00:00:00,4690,1953.90,01-12-2022 00:00:00,4906,1672.60,09-02-2023 00:00:00,4955,1619.75,High,-1.302315,8061.756481,21,1,3
76,INFY,21-08-2023 00:00:00,5084,01-12-2022 00:00:00,4906,1672.60,09-02-2023 00:00:00,4955,1619.75,19-07-2023 00:00:00,5062,1498.80,High,-1.078571,6964.071429,21,1,3
77,INFY,19-10-2023 00:00:00,5125,01-12-2022 00:00:00,4906,1672.60,09-02-2023 00:00:00,4955,1619.75,19-07-2023 00:00:00,5062,1498.80,High,-1.078571,6964.071429,21,1,3


In [126]:
alerts = alerts.drop_duplicates()
alerts.to_excel(alert_file_name,index=False)

In [ ]:
isExist = os.path.exists(alert_file_name)
if(isExist):
    alerts = pd.read_excel(alert_file_name)
else:
    alerts = pd.DataFrame()

In [ ]:
df[df["isPivot"] == 2 ]

In [106]:
kite_authorization_df = pd.read_csv("https://docs.google.com/spreadsheets/d/1EcGGvmEATua3T_t4UskqiDeYRqJcqw-JMHeT_WE51NI/export?gid=0&format=csv")

In [107]:
kite_authorization = kite_authorization_df['Unnamed: 7'][0]
print(kite_authorization)

enctoken Aw8XnMWziDP7SDPyb+86nK/sXGv3GQZR43CYftx/2xzk+KC/ZGw7O+/GdXKJrdIA2IHqRVSgTMQKn6x4fOGUVqJR+f1+LhiZu0vbPuze6Obhg7/xVZu72Q==


In [108]:
maximum_candle_limit = {"minute" : 60,
                         "3minute" : 100,
                         "5minute" : 100,
                          "10minute" : 100,
                          "15minute" : 200,
                          "30minute" : 200,
                          "60minute" : 400,
                          "day" : 2000
                        }

In [109]:
higher_time_frame_data = {"minute" : "day",
                         "3minute" : "day",
                         "5minute" : "day",
                          "10minute" : "day",
                          "15minute" : "day",
                          "30minute" : "week",
                          "60minute" : "week",
                          "day" : "month"
                        }

In [110]:
def get_candle_data(url):
    print(url)
    headers_authorization = kite_authorization
    #headers_authorization = "enctoken uFmgyAvhSeFcXQM/TlH+QvDJs2faxrzlDByU9mVA6KOvDZSybeI5pPNIlzAtoMPsgmN/j+Tx5r/LsqtiTAbtAjVi6gO/A+9iWU3luXSnj0Wwtf11oUkeVg=="
    headers = {
        "authorization": headers_authorization
    }
    response = requests.get(url, headers=headers)
    response.raise_for_status()
    data = response.json()
    #print(data["data"]["candles"])
    candles = data["data"]["candles"]
    candles =  pd.DataFrame(candles)

    if(len(candles)):
      candles.columns = ["Datetime", "Open", "High", "Low", "Close", "Volume","temp"]
      candles.drop(["temp"], axis=1,inplace=True)
      #print(candles)
      candles["prevClose"] = candles['Close'].shift(1)
      candles['day_change'] = ((candles[['prevClose', 'Close']].pct_change(axis=1)['Close']) * 100)
      candles['Year'] = pd.to_datetime(candles["Datetime"]).dt.strftime('%Y').astype(int)
      candles['Month'] = pd.to_datetime(candles["Datetime"]).dt.strftime('%m').astype(int)
      candles['Day'] = pd.to_datetime(candles["Datetime"]).dt.strftime('%d').astype(int)
      candles['Hour'] = pd.to_datetime(candles["Datetime"]).dt.strftime('%H').astype(int)
      candles['Minute'] = pd.to_datetime(candles["Datetime"]).dt.strftime('%M').astype(int)
      candles['Second'] = pd.to_datetime(candles["Datetime"]).dt.strftime('%S').astype(int)
      candles['Week'] = pd.to_datetime(candles["Datetime"]).dt.isocalendar().week
      #print(candles)
      candles["Datetime"] = pd.to_datetime(candles["Datetime"]).dt.strftime('%d-%m-%Y %H:%M:%S')
      candles.set_index("Datetime",inplace = True)

    #print(candles)
    return candles

In [111]:
def string_Formatting(urls,params):
  #print(params)
  formatted_urls = urls.format(**params)
  #print(formatted_urls)
  return formatted_urls

In [112]:
def get_url(rows,is_history_starting_from=False,is_add_indicator=True):
  # print(rows)
  #print(is_history_starting_from,is_add_indicator)
  stock_name = rows['STOCK NAME']
  index_name = rows['INDEX NAME']
  stock_instrument_token = rows['INSTRUMENT_TOKEN']
  fno = rows['FNO']
  # print(stock_name,index_name,stock_instrument_token,fno)
  candles = []
  timeFrame = 'day'
  # timeFrame = '60minute'
  n = maximum_candle_limit[timeFrame]
  url_dates = []

  if(is_history_starting_from):
    current_date = datetime.datetime.now(timezone("Asia/Kolkata"))
    current_date = datetime.datetime(current_date.year,current_date.month,current_date.day)
    start_date = datetime.datetime(2000,1,1)

    while(start_date<current_date):
      start_date_string = start_date.strftime("%Y-%m-%d")
      start_date = start_date + datetime.timedelta(days=n)
      end_date_string = start_date.strftime("%Y-%m-%d")
      url_dates.append([start_date_string,end_date_string])
  else:
    current_date = datetime.datetime.now(timezone("Asia/Kolkata"))
    start_date = current_date - datetime.timedelta(days=n)
    start_date = start_date.strftime("%Y-%m-%d")
    end_date = current_date.strftime("%Y-%m-%d")
    url_dates.append([start_date,end_date])

  #print(url_dates)
  url = "https://kite.zerodha.com/oms/instruments/historical/{stock_instrument_token}/{timeframe}?user_id=TW8928&oi=1&from={start_date}&to={end_date}"
  for url_date in url_dates:
    #print(url_date)
    params = {"timeframe":timeFrame,"stock_instrument_token":stock_instrument_token,"start_date":url_date[0],"end_date":url_date[1]}
    api_url = string_Formatting(url,params.copy())
    #print(api_url)
    candles_data = get_candle_data(api_url)
    # print(candles_data)
    if(len(candles_data)>0):
      candles.append(candles_data)
  #dfs = [df for df in candles]
  candles = pd.concat(candles, axis=0)
  #print(candles)
  candles = candles[~candles.index.duplicated(keep='first')]
  candles.drop_duplicates(inplace=True)
  candles.reset_index(inplace=True)
  candles.insert(1,"Stock name",stock_name)
  candles.insert(2,"Index name",index_name)
  print(candles)
  global df,stockname
  df = candles
  stockname = stock_name
  return candles

In [113]:
stock_data = pd.read_excel("stock market names.xlsx",sheet_name='Stock_list')

In [114]:
stock_data.iloc[48]

STOCK NAME              INFY
INDEX NAME          NIFTY 50
INSTRUMENT_TOKEN      408065
FNO                      FNO
Name: 48, dtype: object

In [ ]:
test1 = ""
threads = []
# is_history_starting_from,is_add_indicator=False,True
is_history_starting_from,is_add_indicator=True,False
for index in range(48,49):#len(stock_data)):
    #print(index)
    row = stock_data.iloc[index]
    #print(row)
    #print(type(row))
    thread = threading.Thread(target=get_url, args=(row,is_history_starting_from,is_add_indicator))
    thread.start()
    threads.append(thread)
    if(index == 48):
      break
for thread in threads:
    thread.join()

In [ ]:
print(df)
print(stockname)

In [ ]:
df[df["isPivot"] == 2 ]

In [ ]:
alerts

In [2]:
def detect_structure(df,stockname,candle, backcandles, window):
    # print(candle)
    # return
    """
    Attention! window should always be greater than the pivot window! to avoid look ahead bias
    """
    try:
        if (candle <= (backcandles+window)) and (candle+window+1 >= len(df)):
            # print(candle)
            return 0
        # if(candle < 5818):
            # return 0
        # print(candle)
        localdf = df.iloc[0:candle-window] 
        Highs = localdf[localdf['isPivot'] == 1].High.tail(5).values
        x_values_Highs = localdf[localdf['isPivot'] == 1].index[-5:].values
        Highs_values = pd.DataFrame(localdf[localdf['isPivot'] == 1].High.tail(5))

        Lows = localdf[localdf['isPivot'] == 2].Low.tail(5).values
        x_values_Lows = localdf[localdf['isPivot'] == 2].index[-5:].values
        Lows_values = pd.DataFrame(localdf[localdf['isPivot'] == 2].Low.tail(5))
        levelbreak = 0
        global alerts
        if Lows_values.shape[0]>=pivot_line_count and candle-window-1 == Lows_values.index[-1]:
            combinations = list(itertools.combinations(zip(x_values_Lows, Lows), pivot_line_count))
            leatest_combinations = [[(index_low[0], index_low[1]) for index_low in combination] for combination in combinations]
            for combination in leatest_combinations:
                slope, intercept , is_line= plus_minus_01_percent(combination,percentage)
                if(is_line):
                    levelbreak = 1
                    alert = pd.DataFrame(combination, columns=['index', 'value']).set_index('index').copy(deep=True)
                    row_to_append = pd.DataFrame({
                        'stockname' : stockname,
                        'alert_date' : [df.iloc[candle].Datetime],
                        'rowNumber' : candle,                    
                        'date1': [df.iloc[alert.index[0]].Datetime],
                        'row1': [alert.index[0]],
                        'value1': [alert.iloc[0, 0]],
                        'date2': [df.iloc[alert.index[1]].Datetime],
                        'row2': [alert.index[1]],
                        'value2': [alert.iloc[1, 0]],
                        'date3': [df.iloc[alert.index[2]].Datetime] if pivot_line_count>2 else 0,
                        'row3': [alert.index[2]] if pivot_line_count>2 else 0,
                        'value3': [alert.iloc[2, 0]] if pivot_line_count>2 else 0,
                        'buyORsell' : 'Low' if levelbreak==1 else 'High',
                        "slope" : slope,
                        "intercept" : intercept,
                        "window_size":window,
                        "percentage_value" : percentage,
                        "pivot_line_count" : pivot_line_count
                    })
                    alerts = pd.concat([alerts, row_to_append], ignore_index=True)
        if Highs_values.shape[0]>=pivot_line_count and candle-window-1 == Highs_values.index[-1]:
            combinations = list(itertools.combinations(zip(x_values_Highs, Highs), pivot_line_count))
            leatest_combinations = [[(index_high[0], index_high[1]) for index_high in combination] for combination in combinations]
            for combination in leatest_combinations:
                slope, intercept , is_line= plus_minus_01_percent(combination,percentage)
                if(is_line):
                    levelbreak = 2
                    alert = pd.DataFrame(combination, columns=['index', 'value']).set_index('index').copy(deep=True)
                    # print(alert)
                    row_to_append = pd.DataFrame({
                        'stockname' : stockname,
                        'alert_date' : [df.iloc[candle].Datetime],
                        'rowNumber' : candle,
                        'date1': [df.iloc[alert.index[0]].Datetime],
                        'row1': [alert.index[0]],
                        'value1': [alert.iloc[0, 0]],
                        'date2': [df.iloc[alert.index[1]].Datetime],
                        'row2': [alert.index[1]],
                        'value2': [alert.iloc[1, 0]],
                        'date3': [df.iloc[alert.index[2]].Datetime] if pivot_line_count>2 else 0,
                        'row3': [alert.index[2]] if pivot_line_count>2 else 0,
                        'value3': [alert.iloc[2, 0]] if pivot_line_count>2 else 0,
                        'buyORsell' : 'Low' if levelbreak==1 else 'High',
                        "slope" : slope,
                        "intercept" : intercept,
                        "window_size":window,
                        "percentage_value" : percentage,
                        "pivot_line_count" : pivot_line_count
                    })
                    # global alerts
                    alerts = pd.concat([alerts, row_to_append], ignore_index=True)
                    # print(alerts)

        # alerts = alerts.drop_duplicates()
        alerts.drop_duplicates(subset=['stockname','date1', 'row1', 'value1', 'date2', 'row2', 'value2', 'date3', 'row3', 'value3', 'buyORsell', 'slope', 'intercept','window_size','percentage_value','pivot_line_count'], keep='last', inplace=True)
        return levelbreak
    except Exception as e:
        # print(e)
        print(f"Error in Detect Structure function: {e}")

In [6]:
def isPivot(df,stock_name,candle, window):
    """
    function that detects if a candle is a pivot/fractal point
    args: candle index, window before and after candle to test if pivot
    returns: 1 if pivot High, 2 if pivot Low, 3 if both and 0 default
    """
    try:
        if candle-window < 0 or candle+window >= len(df):
            return 0
        
        pivotHigh = 1
        pivotLow = 2
        for i in range(candle-window, candle+window+1):
            if df.iloc[candle].Low > df.iloc[i].Low:
                pivotLow=0
            if df.iloc[candle].High < df.iloc[i].High:
                pivotHigh=0
        if (pivotHigh and pivotLow):
            return 3
        elif pivotHigh:
            return pivotHigh
        elif pivotLow:
            return pivotLow
        else:
            return 0
        
    except Exception as e:
        print(f"Error in isPivot function: {e}")

In [7]:
alerts = pd.DataFrame()

In [7]:
stock_name = "INFY"
file_name = f"alerts_day_copy.xlsx"
alerts = pd.read_excel(file_name)
window = 21

In [13]:

candles['isPivot'] = candles.apply(lambda row: isPivot(candles,stock_name,row.name,window), axis=1)
print(candles)
candles['pattern_detected'] = candles.apply(lambda row: detect_structure(candles,stock_name,
                                                        row.name, backcandles=20, window=window,
                                                        ), axis=1)
print(candles)

In [20]:
import logging
logging.basicConfig(filename=f'logfile_test.log',level=logging.INFO, format='%(asctime)s -%(levelname)s - %(message)s')
logging.info(f"Started...")

In [22]:
import traceback
try:
    print("test"+1/0)
except Exception as e:
    traceback_msg = traceback.format_exc()
    print(f"message : {traceback_msg}")

message : Traceback (most recent call last):
  File "C:\Users\Logesh\AppData\Local\Temp\ipykernel_7472\672127929.py", line 3, in <module>
    print("test"+1/0)
                 ~^~
ZeroDivisionError: division by zero



In [8]:
alerts.drop_duplicates(subset=['stockname','date1', 'row1', 'value1', 'date2', 'row2', 'value2', 'date3', 'row3', 'value3', 'buyORsell', 'slope', 'intercept','window_size','percentage_value','pivot_line_count'], keep='last', inplace=True)
alerts.to_excel(file_name,index=False)

In [6]:
def isPivot(df,stock_name,candle, window):
    if candle - window < 0 or candle + window >= len(df):
        return 0

    candle_data = df.iloc[candle]
    window_data = df.iloc[candle - window:candle + window + 1]

    pivotLow = (window_data['Low'] >= candle_data['Low']).all()
    pivotHigh = (window_data['High'] <= candle_data['High']).all()

    if pivotHigh and pivotLow:
        return 3
    elif pivotHigh:
        return 1
    elif pivotLow:
        return 2
    else:
        return 0

In [23]:
def isPivot(df,stock_name,candle, window):
    """
    function that detects if a candle is a pivot/fractal point
    args: candle index, window before and after candle to test if pivot
    returns: 1 if pivot High, 2 if pivot Low, 3 if both and 0 default
    """
    try:
        if candle-window < 0 or candle+window >= len(df):
            return 0
        
        pivotHigh = 1
        pivotLow = 2
        for i in range(candle-window, candle+window+1):
            if df.iloc[candle].Low > df.iloc[i].Low:
                pivotLow=0
            if df.iloc[candle].High < df.iloc[i].High:
                pivotHigh=0
        if (pivotHigh and pivotLow):
            return 3
        elif pivotHigh:
            return pivotHigh
        elif pivotLow:
            return pivotLow
        else:
            return 0
        
    except Exception as e:
        logging.info(f"{stock_name} - Error in isPivot function: {e}")

In [24]:
def two_line_structure(df,stockname,candle, backcandles, window):
    if (candle <= (backcandles+window)) and (candle+window+1 >= len(df)):
        return 0
    localdf = df.iloc[0:candle-window] 
    Highs = localdf[localdf['isPivot'] == 1].High.tail(3).values
    x_values_Highs = localdf[localdf['isPivot'] == 1].index[-3:].values
    Highs_values = pd.DataFrame(localdf[localdf['isPivot'] == 1].High.tail(3))

    Lows = localdf[localdf['isPivot'] == 2].Low.tail(3).values
    x_values_Lows = localdf[localdf['isPivot'] == 2].index[-3:].values
    Lows_values = pd.DataFrame(localdf[localdf['isPivot'] == 2].Low.tail(3))
    levelbreak = 0
    global two_line_alert_df
    if Lows_values.shape[0]>=two_line_count and candle-window-1 == Lows_values.index[-1]:
        combinations = list(itertools.combinations(zip(x_values_Lows, Lows), two_line_count))
        leatest_combinations = [[(index_low[0], index_low[1]) for index_low in combination] for combination in combinations]
        for combination in leatest_combinations:
            x_values,y_values = zip(*combination)
            percent_difference = abs(y_values[0] - y_values[1]) / y_values[1] * 100
            is_line = percent_difference<=1
            if(is_line):
                levelbreak = 1
                alert = pd.DataFrame(combination, columns=['index', 'value']).set_index('index').copy(deep=True)
                row_to_append = pd.DataFrame({
                    'stockname' : stockname,
                    'alert_date' : [df.iloc[candle].Datetime],
                    'rowNumber' : candle,                    
                    'date1': [df.iloc[alert.index[0]].Datetime],
                    'row1': [alert.index[0]],
                    'value1': [alert.iloc[0, 0]],
                    'date2': [df.iloc[alert.index[1]].Datetime],
                    'row2': [alert.index[1]],
                    'value2': [alert.iloc[1, 0]],
                    'buyORsell' : 'Low',
                    "window_size":window,
                    "percentage_value" : percentage,
                    "two_line_count" : two_line_count
                })
                two_line_alert_df = pd.concat([two_line_alert_df, row_to_append], ignore_index=True)

    if Highs_values.shape[0]>=two_line_count and candle-window-1 == Highs_values.index[-1]:
        combinations = list(itertools.combinations(zip(x_values_Highs, Highs), two_line_count))
        leatest_combinations = [[(index_high[0], index_high[1]) for index_high in combination] for combination in combinations]
        for combination in leatest_combinations:
            print(combination)
            x_values,y_values = zip(*combination)
            percent_difference = abs(y_values[0] - y_values[1]) / y_values[1] * 100
            is_line = percent_difference<=1
            if(is_line):
                levelbreak = 2
                alert = pd.DataFrame(combination, columns=['index', 'value']).set_index('index').copy(deep=True)
                row_to_append = pd.DataFrame({
                    'stockname' : stockname,
                    'alert_date' : [df.iloc[candle].Datetime],
                    'rowNumber' : candle,
                    'date1': [df.iloc[alert.index[0]].Datetime],
                    'row1': [alert.index[0]],
                    'value1': [alert.iloc[0, 0]],
                    'date2': [df.iloc[alert.index[1]].Datetime],
                    'row2': [alert.index[1]],
                    'value2': [alert.iloc[1, 0]],
                    'buyORsell' : 'High',
                    "window_size":window,
                    "percentage_value" : percentage,
                    "two_line_count" : two_line_count
                })
                two_line_alert_df = pd.concat([two_line_alert_df, row_to_append], ignore_index=True)
    return levelbreak

In [25]:
alerts = pd.DataFrame()
two_line_count = 2

In [26]:
two_line_file_name = f"two_line_alerts_{time_frame}.xlsx"
two_line_alert_df = pd.DataFrame()

In [27]:
alerts = pd.DataFrame()
candles['two_line_structure'] = candles.apply(lambda row: two_line_structure(candles, stock_name,
                                                                                   row.name, backcandles=20,
                                                                                   window=window),
                                                      axis=1)

[(93, 93.7), (126, 104.45)]
[(93, 93.7), (126, 104.45)]
[(93, 93.7), (168, 115.5)]
[(126, 104.45), (168, 115.5)]
[(126, 104.45), (168, 115.5)]
[(126, 104.45), (197, 119.0)]
[(168, 115.5), (197, 119.0)]
[(168, 115.5), (197, 119.0)]
[(168, 115.5), (246, 239.35)]
[(197, 119.0), (246, 239.35)]
[(197, 119.0), (246, 239.35)]
[(197, 119.0), (362, 257.5)]
[(246, 239.35), (362, 257.5)]
[(246, 239.35), (362, 257.5)]
[(246, 239.35), (397, 377.5)]
[(362, 257.5), (397, 377.5)]
[(362, 257.5), (397, 377.5)]
[(362, 257.5), (435, 33.5)]
[(397, 377.5), (435, 33.5)]
[(397, 377.5), (435, 33.5)]
[(397, 377.5), (487, 42.7)]
[(435, 33.5), (487, 42.7)]
[(435, 33.5), (487, 42.7)]
[(435, 33.5), (533, 40.95)]
[(487, 42.7), (533, 40.95)]
[(487, 42.7), (533, 40.95)]
[(487, 42.7), (602, 34.0)]
[(533, 40.95), (602, 34.0)]
[(533, 40.95), (602, 34.0)]
[(533, 40.95), (648, 33.95)]
[(602, 34.0), (648, 33.95)]
[(602, 34.0), (648, 33.95)]
[(602, 34.0), (675, 37.35)]
[(648, 33.95), (675, 37.35)]
[(648, 33.95), (675, 37.35)

In [28]:
alerts

""


In [5]:
def plus_minus_01_percent(combination,percentage):
    try:
        x_values,y_values = zip(*combination)
        slope, intercept, _, _, _ = linregress(x_values[0:2], y_values[0:2])
        predicted_y_value = slope * x_values[-1] + intercept
        actual_y_value = y_values[-1]
        # print(x_values[-1],y_values[-1],x_values[0:2], y_values[0:2])
        # Calculate 0.1% of the value
        # percent_01 = percentage / 100 * actual_y_value  # 0.1

        percent_difference = abs(predicted_y_value - actual_y_value) / actual_y_value * 100
        # print(percent_difference,percentage)
        return slope, intercept,percent_difference <= percentage

        print(slope, intercept,plus_value,minus_value,actual_y_values,predicted_y_values)
        print(predicted_y_values>minus_value and predicted_y_values < plus_value)
        return slope, intercept,predicted_y_values>minus_value and predicted_y_values < plus_value
    except Exception as e:
        logging.info(f"Error in Plus minus 1 percents function: {e}")
        traceback_msg = traceback.format_exc()
        logging.info(f"message : {traceback_msg}")

In [9]:
window = 10
stock_name = "INFY"
pivot_line_count = 3
percentage = 1
two_line_count  = 2 #3
time_frame = "day"
alert_file_name = f"alerts_{time_frame}_testing.xlsx"
data_store_file_name = f"data_store_{time_frame}.json"
alerts = pd.DataFrame()

In [4]:
candles = pd.read_excel("stock_historical_data\day\ADANIENT.xlsx")

In [32]:
n = 100  # Define the number of last candles to consider

candles['isPivot'] = candles.apply(lambda row: isPivot(candles, stock_name, row.name, window), axis=1)


In [34]:
candles['isPivot_new'] = candles.apply(lambda row: isPivot(candles, stock_name, row.name, window), axis=1)

In [36]:
n = 100  # Define the number of last candles to consider

# Create a backup column
candles['isPivot_backup'] = candles['isPivot_new']

# Apply the function to the last n rows
candles['isPivot_new'] = candles.shift(-n).iloc[-n:].apply(lambda row: isPivot(candles, stock_name, row.name, window), axis=1)

# Fill NaN values with the original values from the backup column
candles['isPivot_new'] = candles['isPivot_new'].fillna(candles['isPivot_backup'])

# Drop the backup column
# candles.drop(columns=['isPivot_backup'], inplace=True)


In [25]:
candles[candles["isPivot"] != candles["isPivot_new"]][["Datetime","Stock name","isPivot","isPivot_new","isPivot_backup"]]

,Datetime,Stock name,isPivot,isPivot_new,isPivot_backup


In [ ]:
n = 100  # Define the number of last candles to consider

candles['pattern_detected'] = candles.apply(lambda row: detect_structure(candles, stock_name,
                                                                                   row.name, backcandles=20,
                                                                                   window=window),
                                                      axis=1)

In [37]:
alerts.drop_duplicates(subset=['stockname','date1', 'row1', 'value1', 'date2', 'row2', 'value2', 'date3', 'row3', 'value3', 'buyORsell', 'slope', 'intercept','window_size','percentage_value','pivot_line_count'], keep='first', inplace=True)
alerts.to_excel(alert_file_name,index=False)

candles.to_excel("stock_historical_data\day\INFY.xlsx",index=False)

In [28]:
candles[candles["pattern_detected"] != 0][["Datetime","Stock name","isPivot","isPivot_new","isPivot_backup","pattern_detected"]]

,Datetime,Stock name,isPivot,isPivot_new,isPivot_backup,pattern_detected
136,2003-07-18,INFY,0,0.0,0.0,1
172,2003-09-09,INFY,1,1.0,1.0,1
201,2003-10-21,INFY,0,0.0,0.0,1
215,2003-11-10,INFY,0,0.0,0.0,2
268,2004-01-27,INFY,2,2.0,2.0,2
...,...,...,...,...,...,...
5176,2024-01-03,INFY,0,0.0,0.0,2
5193,2024-01-29,INFY,0,0.0,0.0,1
5210,2024-02-21,INFY,0,0.0,0.0,2
5216,2024-02-29,INFY,0,0.0,0.0,1


In [29]:
print(candles[candles["isPivot"] !=0][["Datetime","High","Low","isPivot","pattern_detected","two_line_structure","backup"]].to_string())

                Datetime     High      Low  isPivot  pattern_detected  two_line_structure  backup
81   2003-04-30 00:00:00    61.20    56.00        2                 0                   0       0
93   2003-05-19 00:00:00    93.70    89.50        1                 0                   0       0
126  2003-07-03 00:00:00   104.45   100.00        1                 0                   0       0
147  2003-08-01 00:00:00    92.75    90.00        2                 0                   0       0
163  2003-08-26 00:00:00    95.00    90.00        2                 0                   0       0
168  2003-09-02 00:00:00   115.50   106.00        1                 0                   0       0
197  2003-10-14 00:00:00   119.00   109.35        1                 0                   0       0
209  2003-10-29 00:00:00    98.45    95.00        2                 0                   0       0
246  2003-12-19 00:00:00   239.35   211.00        1                 0                   0       0
268  2004-01-21 00:0

In [31]:
two_line_alert_df

,stockname,alert_date,rowNumber,date1,row1,value1,date2,row2,value2,buyORsell,window_size,percentage_value,two_line_count
0,INFY,2003-09-10,174,2003-08-01 00:00:00,147,90.00,2003-08-26,163,90.00,Low,10,1,2
1,INFY,2003-11-13,220,2003-08-01 00:00:00,147,90.00,2003-08-26,163,90.00,Low,10,1,2
2,INFY,2005-01-31,526,2005-01-06 00:00:00,511,35.00,2005-01-12,515,35.00,Low,10,1,2
3,INFY,2005-04-22,583,2005-01-06 00:00:00,511,35.00,2005-01-12,515,35.00,Low,10,1,2
4,INFY,2005-08-08,659,2005-05-19 00:00:00,602,34.00,2005-07-21,648,33.95,High,10,1,2
5,INFY,2005-09-16,686,2005-05-19 00:00:00,602,34.00,2005-07-21,648,33.95,High,10,1,2
6,INFY,2007-06-04,1111,2007-02-07 00:00:00,1033,127.85,2007-05-18,1100,127.00,High,10,1,2
7,INFY,2008-01-18,1271,2007-02-07 00:00:00,1033,127.85,2007-05-18,1100,127.00,High,10,1,2
8,INFY,2012-09-14,2426,2012-08-28 00:00:00,2412,151.50,2012-08-31,2415,151.50,Low,10,1,2
9,INFY,2013-04-15,2569,2012-08-28 00:00:00,2412,151.50,2012-08-31,2415,151.50,Low,10,1,2


In [3]:
url = 'https://catfact.ninja/fact'
url_test_new_list = []
def url_test_new(i):
    session = requests.Session()
    session.cookies.clear()
    try:
        response = session.get(url, 
                               proxies={"http": "http://111.233.225.166:1234"},
                               headers={'User-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36'}
                               timeout = 
                               )
        print(i,end=" ")
    except requests.RequestException as e:
        print(f"An error occurred: {e}")

In [4]:
for index in range(0,100000,100000):
    threads = []
    for index_j in range(index,index+100000):
        thread = threading.Thread(target=url_test_new, args=(index_j,))
        thread.start()
        threads.append(thread)
    for thread in threads:
        thread.join()

12 23 97  41 28 20 55 67 26 74 3 57 29 17 58 24 5 7263  35 77 18 40 36 39 73 27 21 53 11 14 31 2 64 6 38 4 25 8 13 56 0 75 50 10 45 54 51 19 16 32 43 70 47 59 30 1 37 42 44 65 69 62 48 76 68 46 80 34 52 22 60 49 15 71 78 61 8266  33 86 85 79 84 91 94 88 81 87 83 92 89 98 95 96 93 100 90 102 104 101 97 99 103 113 107 106 108 112 110 105 111 117 116 109 118 120 114 115 127 123 122 121 124 119 125 126 133132  130 131 129 135136  139 143 138 128 137 142 134 140 145 147 141 148 149 144 146 156 154 153 150 155 152 151 159 160 157162  161 158 163 166 170 168 164 167 169 165 172 174 176 171 175 178 173 177 181 184 186 183 179 182 188 187 195 185 190 180 193 192 191 189 194 198 196 201 200 199 204 197 205 203 202 208 210 206 209 207 212 214 211 216 215 213 224 217 218 222 223 219 220 225 229 221 228 227 226 230 235 234 232 236 233 231 238 237 243 240 244 245 239 241 246 242 249 248 247 251 252 253 256 250 258 254 257 260 255 262264  259 265 263 267272  261 266 268 270 274 271 269277  273 279276

In [1]:
import logging
import threading
import requests
time_frame = "day"

In [2]:
import logging

# Configure the first logger for INFO level logging
logging.basicConfig(filename=f"logfile_{time_frame}_INFY.log",
                    filemode='a',
                    format='%(asctime)s - %(levelname)s - %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.INFO)

# Create a second logger for DEBUG level logging
logger_debug = logging.getLogger("debug")
handler_debug = logging.FileHandler(filename=f"logfile_{time_frame}_DEBUG.log", mode='a')
formatter_debug = logging.Formatter('%(asctime)s - %(levelname)s - %(message)s', datefmt='%Y-%m-%d %H:%M:%S')
handler_debug.setFormatter(formatter_debug)
logger_debug.addHandler(handler_debug)
logger_debug.setLevel(logging.DEBUG)

# Log an INFO level message to the first logger
logging.info("This is an INFO level message.")

url = 'https://catfact.ninja/fact'
url_test_new_list = []
def url_test_new(i):
    session = requests.Session()
    session.cookies.clear()
    try:
        response = session.get(url, 
                               proxies={"http": "http://111.233.225.166:1234"},
                               headers={'User-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36'}

                               )
        print(i,end=" ")
    except requests.RequestException as e:
        print(f"An error occurred: {e}")
    logging.info(f"{i} - This is a DEBUG level message.")

for index in range(0,10,10):
    threads = []
    for index_j in range(index,index+10):
        thread = threading.Thread(target=url_test_new, args=(index_j,))
        thread.start()
        threads.append(thread)
    for thread in threads:
        thread.join()

1 36 8 0  47  9 52  

In [3]:
url = 'https://catfact.ninja/fact'
url_test_new_list = []
def url_test_new(i):
    session = requests.Session()
    session.cookies.clear()
    try:
        response = session.get(url, 
                               proxies={"http": "http://111.233.225.166:1234"},
                               headers={'User-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/124.0.0.0 Safari/537.36'}

                               )
        print(i,end=" ")
    except requests.RequestException as e:
        print(f"An error occurred: {e}")
    logging.info(f"{i} - This is a DEBUG level message.")

In [4]:
for index in range(0,10,10):
    threads = []
    for index_j in range(index,index+10):
        thread = threading.Thread(target=url_test_new, args=(index_j,))
        thread.start()
        threads.append(thread)
    for thread in threads:
        thread.join()

5 1 0 4 6 2 3 8 7 9 

In [4]:
names = name.split("\n")

In [7]:
n = []
for ns in names:
    # print(ns.split(" ")[0])
    n.append(ns.split(" ")[0])
    # break

In [9]:
len(n)

940

In [12]:
stock_data = pd.read_excel("stock market names.xlsx",sheet_name='Stock_list')

In [13]:
len(stock_data)

637

In [17]:
stock_name_list = []
thread_limit = 25
total_rows = len(stock_data)
for start_index in range(0, total_rows, thread_limit):
    end_index = min(start_index + thread_limit, total_rows)
    for index in range(start_index, end_index):
        row = stock_data.iloc[index]
        stock_name_list.append(row['STOCK NAME'])
        

In [19]:
len(stock_name_list)

637